In [5]:
import pandas as pd
business = pd.read_csv("/Users/xavier/Documents/Masters Y1S2/BAP Project/business_output.csv", low_memory=False)

In [6]:
business.head()           # Show the first 5 rows
business.shape            # Show (rows, columns)
business.columns          # List column names
business.info()           # Overview of the dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 58 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   business_id                            150346 non-null  object 
 1   name                                   150346 non-null  object 
 2   address                                145219 non-null  object 
 3   city                                   150346 non-null  object 
 4   state                                  150346 non-null  object 
 5   postal_code                            150273 non-null  object 
 6   latitude                               150346 non-null  float64
 7   longitude                              150346 non-null  float64
 8   stars                                  150346 non-null  float64
 9   review_count                           150346 non-null  int64  
 10  is_open                                150346 non-null  

In [7]:
#load only the relevant business IDs that are restaurants related
daniel = df = pd.read_json("/Users/xavier/Documents/Masters Y1S2/BAP Project/yelp_academic_dataset_restaurants.json", lines=True)

In [8]:
daniel.head()           # Show the first 5 rows
daniel.shape            # Show (rows, columns)
daniel.columns          # List column names
daniel.info()           # Overview of the dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52268 entries, 0 to 52267
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   52268 non-null  object 
 1   name          52268 non-null  object 
 2   address       52268 non-null  object 
 3   city          52268 non-null  object 
 4   state         52268 non-null  object 
 5   postal_code   52268 non-null  object 
 6   latitude      52268 non-null  float64
 7   longitude     52268 non-null  float64
 8   stars         52268 non-null  float64
 9   review_count  52268 non-null  int64  
 10  is_open       52268 non-null  int64  
 11  attributes    52268 non-null  object 
 12  categories    52268 non-null  object 
 13  hours         52268 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 5.6+ MB


In [9]:
# Fast set for membership tests
daniel_ids = set(daniel['business_id'].unique())

# Sanity checks
print("business rows:", len(business))
print("daniel unique ids:", len(daniel_ids))
print("overlap count:", business['business_id'].isin(daniel_ids).sum())

# Any ids Daniel picked that don't exist in your master?
missing_in_master = daniel_ids.difference(set(business['business_id'].unique()))
print("Missing in master:", len(missing_in_master))
# If you want to inspect a few:
print(list(missing_in_master)[:10])


business rows: 150346
daniel unique ids: 52268
overlap count: 52268
Missing in master: 0
[]


In [10]:
#BUSINESS_SEL
business_sel = business.loc[business['business_id'].isin(daniel_ids)].copy()
print(business_sel.shape)
business_sel.head()


(52268, 58)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,attributes.RestaurantsCounterService,attributes.AgesAllowed,attributes.DietaryRestrictions,hours.Monday,hours.Tuesday,hours.Wednesday,hours.Thursday,hours.Friday,hours.Saturday,hours.Sunday
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,14:0-22:0,16:0-22:0,12:0-22:0,12:0-22:0,12:0-18:0
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,...,NaN,NaN,NaN,0:0-0:0,10:0-18:0,10:0-18:0,10:0-18:0,10:0-18:0,10:0-18:0,12:0-18:0
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,...,NaN,NaN,NaN,9:30-21:30,9:30-21:30,9:30-21:30,9:30-21:30,9:30-21:30,9:30-21:30,10:0-20:0
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,...,NaN,NaN,NaN,11:0-14:0,11:0-14:0,11:0-14:0,11:0-14:0,11:0-14:0,5:0-10:0,15:0-18:0
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,...,NaN,NaN,NaN,7:30-15:30,7:30-15:30,7:30-15:30,7:30-15:30,NaN,NaN,NaN


In [11]:
import pandas as pd
import numpy as np
import ast
import json

# Columns to expand (dict-like text)
DICT_COLS = [
    'attributes.BestNights',
    'attributes.BusinessParking',
    'attributes.HairSpecializesIn',
    'attributes.GoodForMeal',
    'attributes.DietaryRestrictions',
    'attributes.Ambience',
    'attributes.Music',
]

# Parse a cell into a Python dict; keep NaN as NaN
def to_obj_safe(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return np.nan
    if isinstance(x, dict):
        return x
    if isinstance(x, str):
        s = x.strip()
        if not s:
            return np.nan
        try:
            v = ast.literal_eval(s)          # handles "{'key': True}"
            return v if isinstance(v, dict) else np.nan
        except Exception:
            try:
                v = json.loads(s)            # handles '{"key": true}'
                return v if isinstance(v, dict) else np.nan
            except Exception:
                return np.nan
    return np.nan

# Make a safe prefix for new columns
def sanitize_prefix(colname: str) -> str:
    return (colname.replace(" ", "_")
                  .replace(".", "_")
                  .replace("[","").replace("]","")
                  .replace("-","_"))

# Expand multiple dict-like columns
def expand_dict_columns(df: pd.DataFrame, cols):
    df_out = df.copy()
    created = {}

    for col in cols:
        if col not in df_out.columns:
            print(f"[skip] {col} not in dataframe")
            continue

        # 1) parse to dicts (or NaN)
        parsed = df_out[col].map(to_obj_safe)

        # 2) normalize dicts to columns (only non-NaN rows), then reindex
        expanded = pd.json_normalize(parsed.dropna())
        expanded = expanded.reindex(parsed.index)

        # 3) prefix new columns for clarity
        prefix = sanitize_prefix(col) + "__"
        expanded.columns = [prefix + str(c).replace(" ", "_") for c in expanded.columns]

        # 4) keep NaN rows as NaN across all expanded cols
        expanded.loc[parsed.isna(), :] = np.nan

        # 5) replace original column with expanded columns
        df_out = pd.concat([df_out.drop(columns=[col]), expanded], axis=1)

        created[col] = list(expanded.columns)

    return df_out, created

# ===== RUN IT on your filtered 58-col dataframe =====
# (make sure 'business_sel' exists from earlier filtering by Daniel's IDs)
df_expanded, new_cols_map = expand_dict_columns(business_sel, DICT_COLS)

# Quick peek
print("New shape:", df_expanded.shape)
for k, v in new_cols_map.items():
    print(f"{k} → {len(v)} new cols (e.g., {v[:5]})")

# First rows
df_expanded.head()


New shape: (52268, 100)
attributes.BestNights → 7 new cols (e.g., ['attributes_BestNights__monday', 'attributes_BestNights__tuesday', 'attributes_BestNights__friday', 'attributes_BestNights__wednesday', 'attributes_BestNights__thursday'])
attributes.BusinessParking → 5 new cols (e.g., ['attributes_BusinessParking__garage', 'attributes_BusinessParking__street', 'attributes_BusinessParking__validated', 'attributes_BusinessParking__lot', 'attributes_BusinessParking__valet'])
attributes.HairSpecializesIn → 8 new cols (e.g., ['attributes_HairSpecializesIn__straightperms', 'attributes_HairSpecializesIn__coloring', 'attributes_HairSpecializesIn__extensions', 'attributes_HairSpecializesIn__africanamerican', 'attributes_HairSpecializesIn__curly'])
attributes.GoodForMeal → 6 new cols (e.g., ['attributes_GoodForMeal__dessert', 'attributes_GoodForMeal__latenight', 'attributes_GoodForMeal__lunch', 'attributes_GoodForMeal__dinner', 'attributes_GoodForMeal__brunch'])
attributes.DietaryRestrictions → 

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,attributes_Ambience__upscale,attributes_Ambience__classy,attributes_Ambience__casual,attributes_Music__dj,attributes_Music__background_music,attributes_Music__no_music,attributes_Music__jukebox,attributes_Music__live,attributes_Music__video,attributes_Music__karaoke
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,...,False,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_expanded.head()
df_expanded.info()
df_expanded[['attributes_Music__dj',
             'attributes_Music__background_music',
             'attributes_Music__no_music']].head(10)


<class 'pandas.core.frame.DataFrame'>
Index: 52268 entries, 3 to 150340
Data columns (total 100 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   business_id                                    52268 non-null  object 
 1   name                                           52268 non-null  object 
 2   address                                        51825 non-null  object 
 3   city                                           52268 non-null  object 
 4   state                                          52268 non-null  object 
 5   postal_code                                    52247 non-null  object 
 6   latitude                                       52268 non-null  float64
 7   longitude                                      52268 non-null  float64
 8   stars                                          52268 non-null  float64
 9   review_count                                   52268 

,attributes_Music__dj,attributes_Music__background_music,attributes_Music__no_music
3,NaN,NaN,NaN
5,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN
11,NaN,NaN,NaN
12,NaN,NaN,NaN
14,NaN,NaN,NaN
15,NaN,NaN,NaN
19,NaN,NaN,NaN
20,NaN,NaN,NaN


In [13]:
import numpy as np
import pandas as pd

# prefixes to audit (your expanded attribute families)
prefixes = [
    "attributes_BestNights__",
    "attributes_BusinessParking__",
    "attributes_HairSpecializesIn__",
    "attributes_GoodForMeal__",
    "attributes_DietaryRestrictions__",
    "attributes_Ambience__",
    "attributes_Music__",
]

cols_to_audit = [c for c in df_expanded.columns if any(c.startswith(p) for p in prefixes)]

def audit_boolish_columns(df, columns):
    rows = []
    for c in columns:
        s = df[c]

        blank_cnt = int(s.map(lambda v: isinstance(v, str) and v.strip() == "").sum())

        nonna = s[~s.isna()]
        invalid_cnt = int((~nonna.isin([True, False])).sum())

        n_true  = int((s == True).sum())
        n_false = int((s == False).sum())
        n_nan   = int(s.isna().sum())
        n_nonnull = len(s) - n_nan
        pct_true = (n_true / n_nonnull) if n_nonnull else np.nan

        rows.append({
            "group": next(g for g in ["BestNights","BusinessParking","HairSpecializesIn",
                                      "GoodForMeal","DietaryRestrictions","Ambience","Music"]
                          if c.startswith(f"attributes_{g}__")),
            "key": c.split("__", 1)[1] if "__" in c else c,
            "column": c,
            "dtype": str(s.dtype),
            "True": n_true,
            "False": n_false,
            "NaN": n_nan,
            "non_null": n_nonnull,
            "pct_true_among_non_null": round(pct_true, 4) if pd.notna(pct_true) else np.nan,
            "blank_strings": blank_cnt,
            "invalid_nonblank": invalid_cnt,
        })

    out = (pd.DataFrame(rows)
             .sort_values(["group","key"])
             .reset_index(drop=True))
    return out

audit_df = audit_boolish_columns(df_expanded, cols_to_audit)

# Show a clean DataFrame (copy-friendly in Jupyter)
display_cols = ["group","key","column","dtype","True","False","NaN",
                "non_null","pct_true_among_non_null","blank_strings","invalid_nonblank"]

# show all rows without truncation (optional)
pd.set_option("display.max_rows", None)

audit_df[display_cols]


,group,key,column,dtype,True,False,NaN,non_null,pct_true_among_non_null,blank_strings,invalid_nonblank
0,Ambience,casual,attributes_Ambience__casual,object,810,741,50717,1551,0.5222,0,0
1,Ambience,classy,attributes_Ambience__classy,object,214,1271,50783,1485,0.1441,0,0
2,Ambience,divey,attributes_Ambience__divey,object,56,1347,50865,1403,0.0399,0,0
3,Ambience,hipster,attributes_Ambience__hipster,object,47,1429,50792,1476,0.0318,0,0
4,Ambience,intimate,attributes_Ambience__intimate,object,38,1433,50797,1471,0.0258,0,0
5,Ambience,romantic,attributes_Ambience__romantic,object,22,1475,50771,1497,0.0147,0,0
6,Ambience,touristy,attributes_Ambience__touristy,object,17,1466,50785,1483,0.0115,0,0
7,Ambience,trendy,attributes_Ambience__trendy,object,106,1323,50839,1429,0.0742,0,0
8,Ambience,upscale,attributes_Ambience__upscale,object,16,1500,50752,1516,0.0106,0,0
9,BestNights,friday,attributes_BestNights__friday,object,19,12,52237,31,0.6129,0,0


In [14]:
# 0) If you don't already have it:
# audit_df = (the table we just built)

# 1) Global check: are there any blanks or invalids anywhere?
only_true_false_nan = (audit_df['blank_strings'].sum() == 0) & (audit_df['invalid_nonblank'].sum() == 0)
print("Only True/False/NaN across ALL audited columns:", only_true_false_nan)

# 2) Per-column invariant: non_null should equal True + False
#    (since blank/invalid are zero if the first check passed)
mismatch_cols = audit_df.loc[
    audit_df['non_null'] != (audit_df['True'] + audit_df['False']),
    'column'
].tolist()
print("Columns where non_null != True+False:", mismatch_cols)


Only True/False/NaN across ALL audited columns: True
Columns where non_null != True+False: []


In [15]:
#cast to proper nullable booleans
# which columns to cast (the audited ones)
prefixes = [
    "attributes_BestNights__",
    "attributes_BusinessParking__",
    "attributes_HairSpecializesIn__",
    "attributes_GoodForMeal__",
    "attributes_DietaryRestrictions__",
    "attributes_Ambience__",
    "attributes_Music__",
]
flag_cols = [c for c in df_expanded.columns if any(c.startswith(p) for p in prefixes)]

# convert to pandas nullable Boolean: True / False / <NA>
for c in flag_cols:
    df_expanded[c] = df_expanded[c].astype("boolean")


In [16]:
import pandas as pd
import numpy as np

# ✅ The expanded flag families we care about
prefixes = [
    "attributes_BestNights__",
    "attributes_BusinessParking__",
    "attributes_HairSpecializesIn__",
    "attributes_GoodForMeal__",
    "attributes_DietaryRestrictions__",
    "attributes_Ambience__",
    "attributes_Music__",
]

# 1) Collect the flag columns to summarize
flag_cols = [c for c in df_expanded.columns if any(c.startswith(p) for p in prefixes)]

# 2) Build a long-form summary table
rows = []
for col in flag_cols:
    s = df_expanded[col]
    n_true = int((s == True).sum())
    n_false = int((s == False).sum())
    n_nan = int(s.isna().sum())
    n_nonnull = len(s) - n_nan
    pct_true = (n_true / n_nonnull) if n_nonnull else np.nan

    # infer group name from the prefix
    group = next(g for g in [
        "BestNights","BusinessParking","HairSpecializesIn",
        "GoodForMeal","DietaryRestrictions","Ambience","Music"
    ] if col.startswith(f"attributes_{g}__"))

    rows.append({
        "group": group,
        "key": col.split("__", 1)[1],               # part after the double underscore
        "column": col,                               # full column name
        "dtype": str(s.dtype),
        "True": n_true,
        "False": n_false,
        "NaN": n_nan,
        "non_null": n_nonnull,
        "pct_true_among_non_null": round(pct_true, 4) if pd.notna(pct_true) else np.nan,
    })

attr_summary = (pd.DataFrame(rows)
                .sort_values(["group","key"])
                .reset_index(drop=True))

# 3) Preview in notebook (pretty table)
attr_summary.head(20)

# 4) Save to CSV for Excel / further EDA
attr_summary.to_csv("attribute_boolean_summary.csv", index=False)
print("Saved to attribute_boolean_summary.csv")


Saved to attribute_boolean_summary.csv


In [17]:
import pandas as pd
import numpy as np

# ✅ Columns you listed (non-nested flags)
SIMPLE_ATTRS = [
    "attributes.ByAppointmentOnly",
    "attributes.BusinessAcceptsCreditCards",
    "attributes.BikeParking",
    "attributes.CoatCheck",
    "attributes.RestaurantsTakeOut",
    "attributes.RestaurantsDelivery",
    "attributes.Caters",
    "attributes.WheelchairAccessible",
    "attributes.OutdoorSeating",
    "attributes.HasTV",
    "attributes.RestaurantsReservations",
    "attributes.DogsAllowed",
    "attributes.GoodForKids",
    "attributes.RestaurantsTableService",
    "attributes.RestaurantsGoodForGroups",
    "attributes.DriveThru",
    "attributes.BusinessAcceptsBitcoin",
    "attributes.AcceptsInsurance",
    "attributes.BYOB",
    "attributes.Corkage",
    "attributes.BYOBCorkage",
    "attributes.Open24Hours",
    "attributes.RestaurantsCounterService",
]

# Helper: coerce to True/False/<NA> robustly
def to_bool_na(v):
    if v is None or (isinstance(v, float) and np.isnan(v)):
        return pd.NA
    if isinstance(v, bool):
        return v
    if isinstance(v, str):
        t = v.strip().lower()
        if t in {"true", "t", "yes", "y", "1"}:
            return True
        if t in {"false", "f", "no", "n", "0"}:
            return False
        if t in {"", "na", "n/a", "nan", "none", "null"}:
            return pd.NA
    return pd.NA  # anything else -> unknown

rows = []
actually_casted = []

for col in SIMPLE_ATTRS:
    if col not in df_expanded.columns:
        # skip cleanly if a column doesn't exist
        rows.append({
            "column": col, "dtype": "MISSING",
            "True": 0, "False": 0, "NaN": 0, "non_null": 0, "pct_true_among_non_null": np.nan
        })
        continue

    s = df_expanded[col].map(to_bool_na).astype("boolean")  # cast to nullable boolean
    df_expanded[col] = s  # keep the cleaned version in your dataframe
    actually_casted.append(col)

    n_true = int((s == True).sum())
    n_false = int((s == False).sum())
    n_nan = int(s.isna().sum())
    n_nonnull = len(s) - n_nan
    pct_true = (n_true / n_nonnull) if n_nonnull else np.nan

    rows.append({
        "column": col,
        "dtype": str(s.dtype),
        "True": n_true,
        "False": n_false,
        "NaN": n_nan,
        "non_null": n_nonnull,
        "pct_true_among_non_null": round(pct_true, 4) if pd.notna(pct_true) else np.nan,
    })

simple_attr_summary = (
    pd.DataFrame(rows)
      .sort_values("column")
      .reset_index(drop=True)
)

# 👀 Pretty table in notebook
simple_attr_summary

# (Optional) Save to CSV for Excel/EDA
# simple_attr_summary.to_csv("simple_attribute_boolean_summary.csv", index=False)

# (Optional) quick sanity check: all are only True/False/<NA>
only_tfn = (simple_attr_summary["non_null"] == (simple_attr_summary["True"] + simple_attr_summary["False"])).all()
print("Only True/False/<NA> for all casted simple attributes:", bool(only_tfn))


Only True/False/<NA> for all casted simple attributes: True


In [18]:
simple_attr_summary.to_csv("simple_attribute_boolean_summary.csv", index=False)


In [19]:
# Look at the *original* strings for this field
business_sel['attributes.AcceptsInsurance'].value_counts(dropna=False).head(20)


attributes.AcceptsInsurance
NaN      50314
True      1358
False      596
Name: count, dtype: int64

In [20]:
# Recompute normalization *just for inspection* (doesn't overwrite anything)
def to_bool_na_loose(v):
    import numpy as np, pandas as pd
    if v is None or (isinstance(v, float) and np.isnan(v)): return pd.NA
    if isinstance(v, bool): return v
    if isinstance(v, str):
        t = v.strip().lower()
        if t in {"true","t","yes","y","1"}:  return True
        if t in {"false","f","no","n","0"}:  return False
        if t in {"","na","n/a","nan","none","null"}: return pd.NA
    return pd.NA

tmp = pd.DataFrame({"raw": business_sel["attributes.AcceptsInsurance"]})
tmp["normalized"] = tmp["raw"].map(to_bool_na_loose)

# Cross-tab of raw → normalized
pd.crosstab(tmp["raw"], tmp["normalized"], dropna=False)


normalized,False,True,NaN
raw,,,
NaN,0,0,50314
False,596,0,0
True,0,1358,0


In [21]:
#sorry i missed out 2
import pandas as pd
import numpy as np

# New simple (non-nested) flags you missed
NEW_SIMPLE_ATTRS = [
    "attributes.HappyHour",
    "attributes.GoodForDancing",
]

# Robust coercion: 'True'/'False'/yes/no/1/0 -> True/False, blanks -> <NA>
def to_bool_na(v):
    if v is None or (isinstance(v, float) and np.isnan(v)): return pd.NA
    if isinstance(v, bool): return v
    if isinstance(v, str):
        t = v.strip().lower()
        if t in {"true","t","yes","y","1"}:  return True
        if t in {"false","f","no","n","0"}:  return False
        if t in {"","na","n/a","nan","none","null"}: return pd.NA
    return pd.NA

rows = []
for col in NEW_SIMPLE_ATTRS:
    if col not in df_expanded.columns:
        rows.append({"column": col, "dtype": "MISSING",
                     "True": 0, "False": 0, "NaN": 0,
                     "non_null": 0, "pct_true_among_non_null": np.nan})
        continue

    s = df_expanded[col].map(to_bool_na).astype("boolean")   # cast to True/False/<NA>
    df_expanded[col] = s                                     # keep cleaned version

    n_true = int((s == True).sum())
    n_false = int((s == False).sum())
    n_nan = int(s.isna().sum())
    n_nonnull = len(s) - n_nan
    pct_true = (n_true / n_nonnull) if n_nonnull else np.nan

    rows.append({
        "column": col,
        "dtype": str(s.dtype),
        "True": n_true,
        "False": n_false,
        "NaN": n_nan,
        "non_null": n_nonnull,
        "pct_true_among_non_null": round(pct_true, 4) if pd.notna(pct_true) else np.nan,
    })

new_simple_summary = (pd.DataFrame(rows)
                      .sort_values("column")
                      .reset_index(drop=True))

# Show the pretty table
new_simple_summary


,column,dtype,True,False,NaN,non_null,pct_true_among_non_null
0,attributes.GoodForDancing,boolean,293,1291,50684,1584,0.1850
1,attributes.HappyHour,boolean,3342,1881,47045,5223,0.6399


In [22]:
#now for those not true/false ones
import pandas as pd
import numpy as np
import ast

# Categorical columns you’re summarizing
CATEGORICAL_ATTRS = [
    "attributes.RestaurantsPriceRange2",  # 1..4
    "attributes.Alcohol",                 # none, full_bar, beer_and_wine
    "attributes.RestaurantsAttire",       # casual, dressy, formal
    "attributes.NoiseLevel",              # quiet, average, loud, very_loud
    "attributes.Smoking",                 # no, yes, outdoor
]

def norm_cat(v):
    """Dequote python-ish strings like u'full_bar', then normalize."""
    # Missing
    if v is None or (isinstance(v, float) and np.isnan(v)):
        return pd.NA

    # Try to dequote strings like "u'full_bar'" or "'beer_and_wine'"
    if isinstance(v, str):
        s = v.strip()
        try:
            parsed = ast.literal_eval(s)  # "u'full_bar'" -> "full_bar"
            if isinstance(parsed, str):
                s = parsed
        except Exception:
            # not a python literal; keep original s
            pass

        t = s.strip().lower()
        if t in {"", "na", "n/a", "nan", "null"}:
            return pd.NA
        t = t.replace("&", "and").replace(" ", "_")
        return t

    # Non-string (e.g., number for price range) -> keep as-is
    return v

# Clean the columns in-place and build a fresh summary
rows = []
for col in CATEGORICAL_ATTRS:
    if col not in df_expanded.columns:
        rows.append({"column": col, "category": "<missing column>", "count": 0,
                     "non_null": 0, "pct_of_non_null": np.nan})
        continue

    s = df_expanded[col].map(norm_cat)

    # Special: price range as nullable integer 1..4
    if col == "attributes.RestaurantsPriceRange2":
        s = pd.to_numeric(s, errors="coerce").astype("Int64")
    else:
        # compact memory + tidy type
        s = s.astype("category")

    # Write cleaned back to dataframe
    df_expanded[col] = s

    # Counts
    n_nan = int(s.isna().sum())
    non_null = len(s) - n_nan

    vc = (s.dropna()
            .value_counts()
            .sort_index())

    for cat, cnt in vc.items():
        rows.append({
            "column": col,
            "category": cat,
            "count": int(cnt),
            "non_null": int(non_null),
            "pct_of_non_null": round(cnt / non_null, 4) if non_null else np.nan,
        })

    # Add NA row for visibility
    rows.append({
        "column": col,
        "category": "<NA>",
        "count": n_nan,
        "non_null": int(non_null),
        "pct_of_non_null": np.nan,
    })

cat_summary = (pd.DataFrame(rows)
               .sort_values(["column","category"])
               .reset_index(drop=True))

# 👀 Pretty table (now without the u'...' duplicates)
cat_summary


,column,category,count,non_null,pct_of_non_null
0,attributes.Alcohol,<NA>,37417,14851,NaN
1,attributes.Alcohol,beer_and_wine,2127,14851,0.1432
2,attributes.Alcohol,full_bar,5573,14851,0.3753
3,attributes.Alcohol,none,7151,14851,0.4815
4,attributes.NoiseLevel,<NA>,39148,13120,NaN
5,attributes.NoiseLevel,average,9078,13120,0.6919
6,attributes.NoiseLevel,loud,1026,13120,0.0782
7,attributes.NoiseLevel,quiet,2590,13120,0.1974
8,attributes.NoiseLevel,very_loud,426,13120,0.0325
9,attributes.RestaurantsAttire,<NA>,38722,13546,NaN


In [23]:
# WHY IS DIETARY RESTRICTIONS ALL NAN?
1) Confirm the source column is entirely missing in your subset
business_sel['attributes.DietaryRestrictions'].notna().sum()
# -> should print 0

# 2) Double-check the expanded flags really have no data
diet_cols = [c for c in df_expanded.columns if c.startswith('attributes_DietaryRestrictions__')]
{c: int((df_expanded[c] == True).sum() + (df_expanded[c] == False).sum()) for c in diet_cols}
# -> all zeros

# 3) Sanity check: are these rows actually restaurants?
is_rest = business_sel['categories'].fillna('').str.contains('Restaurants', case=False)
is_rest.sum(), len(business_sel)
# -> shows how many of your 52,268 have "Restaurants" in categories

b# 4) Even among restaurants, coverage is still zero?
business_sel.loc[is_rest, 'attributes.DietaryRestrictions'].notna().sum()
# -> should still be 0


SyntaxError: unmatched ')' (159476396.py, line 2)

In [ ]:
import pandas as pd, numpy as np, ast, json, re

def to_obj_safe_plus(x):
    """Parse dicts, lists, or single strings into a dict of {key: True}.
       Returns NaN if nothing usable."""
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return np.nan
    if isinstance(x, dict):
        return x
    if isinstance(x, list):
        return {str(k).strip().lower().replace(" ", "-"): True for k in x if str(k).strip()}
    if isinstance(x, str):
        s = x.strip()
        if not s:
            return np.nan
        # Try Python literal first (handles "u'...'", "['a','b']", "{'a': True}")
        try:
            v = ast.literal_eval(s)
            if isinstance(v, dict):
                return v
            if isinstance(v, list):
                return {str(k).strip().lower().replace(" ", "-"): True for k in v if str(k).strip()}
            if isinstance(v, str):
                token = v.strip()
                return {token.lower().replace(" ", "-"): True} if token else np.nan
        except Exception:
            pass
        # Try JSON
        try:
            v = json.loads(s)
            if isinstance(v, dict):
                return v
            if isinstance(v, list):
                return {str(k).strip().lower().replace(" ", "-"): True for k in v if str(k).strip()}
        except Exception:
            pass
        # Fallback: comma/semicolon/slash separated string
        parts = [p.strip() for p in re.split(r"[;,/]", s) if p.strip()]
        if parts:
            return {p.lower().replace(" ", "-"): True for p in parts}
        return np.nan
    return np.nan

def sanitize_prefix(colname: str) -> str:
    return (colname.replace(" ", "_")
                  .replace(".", "_")
                  .replace("[","").replace("]","")
                  .replace("-","_"))

# --- Re-expand just DietaryRestrictions using the upgraded parser ---
source_col = "attributes.DietaryRestrictions"

# Parse from the ORIGINAL dataframe (business_sel still has the raw column)
parsed = business_sel[source_col].map(to_obj_safe_plus)

expanded = pd.json_normalize(parsed.dropna())
expanded = expanded.reindex(business_sel.index)

prefix = sanitize_prefix(source_col) + "__"
expanded.columns = [prefix + str(c) for c in expanded.columns]

# Keep NaN rows as NaN across all expanded cols
expanded.loc[parsed.isna(), :] = np.nan

# Replace any prior DietaryRestrictions expanded columns in df_expanded
old_cols = [c for c in df_expanded.columns if c.startswith("attributes_DietaryRestrictions__")]
df_expanded = pd.concat([df_expanded.drop(columns=old_cols, errors="ignore"), expanded], axis=1)

# Cast to nullable booleans
for c in expanded.columns:
    df_expanded[c] = df_expanded[c].astype("boolean")

# Quick sanity check: show the new columns and non-null counts
print("New DietaryRestrictions keys:", list(expanded.columns))
print("Non-null per key:", df_expanded[expanded.columns].notna().sum().to_dict())


In [ ]:
rows = business_sel.loc[
    business_sel['attributes.DietaryRestrictions'].notna(),
    ['business_id','name','attributes.DietaryRestrictions']
]
rows.head(20)


In [ ]:
import pandas as pd, numpy as np, ast, json, re

src = "attributes.DietaryRestrictions"

def to_obj_safe_plus(x):
    """Parse dict/list/string into {key: True}; treat empties as missing."""
    if x is None or (isinstance(x, float) and np.isnan(x)): return np.nan
    if isinstance(x, str):
        s = x.strip()
        if not s or s.lower() in {"none","null","na","n/a"} or s in {"[]","{}"}:
            return np.nan
        try:
            v = ast.literal_eval(s)
        except Exception:
            v = s
        x = v
    if x is None: return np.nan
    if isinstance(x, dict):   return x if len(x) else np.nan
    if isinstance(x, list):   return {str(k).strip().lower().replace(" ", "-"): True
                                      for k in x if str(k).strip()} or np.nan
    if isinstance(x, str):
        token = x.strip().strip("'\"")
        if not token or token.lower() in {"none","null"}: return np.nan
        return {token.lower().replace(" ", "-"): True}
    return np.nan

def sanitize_prefix(colname: str) -> str:
    return (colname.replace(" ", "_")
                  .replace(".", "_")
                  .replace("[","").replace("]","")
                  .replace("-","_"))

# 1) Parse from the ORIGINAL df that still has the raw string column
parsed = business_sel[src].map(to_obj_safe_plus)

# 2) Normalize only non-missing rows
exp = pd.json_normalize(parsed.dropna())
if not exp.empty:
    # 3) carry business_id for a safe key-merge
    exp["business_id"] = business_sel.loc[exp.index, "business_id"].values

    # 4) prefix keys
    prefix = sanitize_prefix(src) + "__"
    exp = exp.rename(columns=lambda c: prefix + str(c) if c != "business_id" else c)

    # 5) remove any old DietaryRestrictions expanded cols, then LEFT MERGE by key
    old = [c for c in df_expanded.columns if c.startswith("attributes_DietaryRestrictions__")]
    df_expanded = df_expanded.drop(columns=old, errors="ignore").merge(exp, on="business_id", how="left")

    # 6) cast to nullable booleans
    new_cols = [c for c in df_expanded.columns if c.startswith("attributes_DietaryRestrictions__")]
    for c in new_cols:
        df_expanded[c] = df_expanded[c].astype("boolean")

    # 7) sanity checks
    print("New cols:", new_cols)
    print("True counts:", {c: int((df_expanded[c] == True).sum()) for c in new_cols})
    print("False counts:", {c: int((df_expanded[c] == False).sum()) for c in new_cols})
    print("Non-null:", df_expanded[new_cols].notna().sum().to_dict())
else:
    print("No parsable DietaryRestrictions found.")


In [ ]:
import pandas as pd, numpy as np, ast, json, re

src = "attributes.DietaryRestrictions"

def to_obj_safe_plus(x):
    """Parse dict/list/string into {key: True}; treat empties as missing."""
    if x is None or (isinstance(x, float) and np.isnan(x)): return np.nan
    if isinstance(x, str):
        s = x.strip()
        if not s or s.lower() in {"none","null","na","n/a"} or s in {"[]","{}"}:
            return np.nan
        try:
            v = ast.literal_eval(s)
        except Exception:
            v = s
        x = v
    if x is None: return np.nan
    if isinstance(x, dict):   return x if len(x) else np.nan
    if isinstance(x, list):   return ({str(k).strip().lower().replace(" ", "-"): True
                                       for k in x if str(k).strip()} or np.nan)
    if isinstance(x, str):
        token = x.strip().strip("'\"")
        if not token or token.lower() in {"none","null"}: return np.nan
        return {token.lower().replace(" ", "-"): True}
    return np.nan

def sanitize_prefix(colname: str) -> str:
    return (colname.replace(" ", "_")
                  .replace(".", "_")
                  .replace("[","").replace("]","")
                  .replace("-","_"))

# 1) Parse the ORIGINAL raw column
parsed = business_sel[src].map(to_obj_safe_plus)

# 2) Keep only rows that parsed to something, and carry business_id with them
tmp = business_sel.loc[parsed.notna(), ["business_id"]].copy()
tmp["__parsed__"] = parsed.loc[parsed.notna()].values

# 3) Normalize the dicts to columns (index-free), then attach business_id
exp_keys = pd.json_normalize(tmp["__parsed__"]).reset_index(drop=True)
exp      = pd.concat([tmp[["business_id"]].reset_index(drop=True), exp_keys], axis=1)

# 4) Prefix the new columns
prefix = sanitize_prefix(src) + "__"
exp = exp.rename(columns=lambda c: c if c=="business_id" else prefix + str(c))

# 5) Remove any old DietaryRestrictions columns, then MERGE by business_id
old_cols = [c for c in df_expanded.columns if c.startswith("attributes_DietaryRestrictions__")]
df_expanded = df_expanded.drop(columns=old_cols, errors="ignore").merge(exp, on="business_id", how="left")

# 6) Cast to pandas nullable booleans
new_cols = [c for c in df_expanded.columns if c.startswith("attributes_DietaryRestrictions__")]
for c in new_cols:
    df_expanded[c] = df_expanded[c].astype("boolean")

# 7) Sanity checks
print("New DietaryRestrictions columns:", new_cols)
print("True counts:",  {c: int((df_expanded[c] == True).sum())  for c in new_cols})
print("False counts:", {c: int((df_expanded[c] == False).sum()) for c in new_cols})
print("Non-null:",     df_expanded[new_cols].notna().sum().to_dict())


In [ ]:
ids = ["2iRMSBYN8F51N7znXEu4TQ","AroThfwhNmj8YNIQePGwlQ","zXgksmhdzgL4Xx5FylbP3w"]
cols = [c for c in df_expanded.columns if c.startswith("attributes_DietaryRestrictions__")]
df_expanded.loc[df_expanded["business_id"].isin(ids), ["business_id","name"] + cols]


In [ ]:
# Coverage per key (should all be 9 here)
diet_cols = [c for c in df_expanded.columns if c.startswith("attributes_DietaryRestrictions__")]
TOTAL = len(df_expanded)
{c: (df_expanded[c].notna().sum(), round(df_expanded[c].notna().mean(), 6)) for c in diet_cols}
# -> (count, coverage_fraction)

# Optional: a single indicator that the business reports *any* dietary info
df_expanded["has_dietary_info"] = df_expanded[diet_cols].notna().any(axis=1)
df_expanded["has_dietary_info"] = df_expanded["has_dietary_info"].astype("boolean")

# If you decide to drop the per-key columns
df_expanded = df_expanded.drop(columns=diet_cols)


In [ ]:
import pandas as pd
import numpy as np

# ========= 1) basic shape / identity =========
print("Rows, columns:", df_expanded.shape)
print("business_id unique:", df_expanded["business_id"].is_unique)
print("Missing business_id:", int(df_expanded["business_id"].isna().sum()))

# Approx memory use
mem_mb = df_expanded.memory_usage(deep=True).sum() / 1024**2
print(f"Approx memory: {mem_mb:.1f} MB")

# Dtype mix
print("\nDtypes count:\n", df_expanded.dtypes.astype(str).value_counts())

# Quick peek
display(df_expanded.head(3))
display(df_expanded.tail(3))

# ========= 2) what got expanded (families & coverage) =========
# All expanded attribute columns have the pattern attributes_Family__key
exp_cols = [c for c in df_expanded.columns if c.startswith("attributes_") and "__" in c]

def split_fam_key(col):
    left, right = col.split("__", 1)
    fam = left.replace("attributes_", "")
    return fam, right

families = pd.Series([split_fam_key(c)[0] for c in exp_cols]).value_counts()
print("\nExpanded families (number of columns per family):")
display(families.to_frame("num_cols"))

# Family-level coverage: % of rows that have *any* value in that family
fam_cov_rows = []
for fam in families.index:
    fam_cols = [c for c in exp_cols if split_fam_key(c)[0] == fam]
    any_present = df_expanded[fam_cols].notna().any(axis=1).mean()
    fam_cov_rows.append({"family": fam, "rows_with_any_value": int(any_present*len(df_expanded)),
                         "coverage_pct": round(any_present*100, 2)})
fam_cov = pd.DataFrame(fam_cov_rows).sort_values("coverage_pct", ascending=False)
print("\nFamily-level coverage (any key present):")
display(fam_cov)

# ========= 3) per-column non-null & booleans true/false =========
# Build a tidy summary for expanded columns
rows = []
for c in exp_cols:
    s = df_expanded[c]
    non_null = int(s.notna().sum())
    na = int(s.isna().sum())
    dtype = str(s.dtype)
    rec = {"column": c, "dtype": dtype, "non_null": non_null, "NaN": na,
           "coverage_pct": round(non_null/len(df_expanded)*100, 2)}
    if dtype == "boolean":
        rec["TRUE"]  = int((s == True).sum())
        rec["FALSE"] = int((s == False).sum())
    rows.append(rec)

col_summary = (pd.DataFrame(rows)
               .sort_values(["coverage_pct","non_null","column"], ascending=[False, False, True])
               .reset_index(drop=True))

print("\nTop expanded columns by coverage:")
display(col_summary.head(20))

# Columns that are all missing (sanity)
all_missing = col_summary[col_summary["non_null"] == 0]["column"].tolist()
print(f"\nExpanded columns with 0 non-null (count={len(all_missing)}):")
print(all_missing[:20], "...")  # show a few

# ========= 4) quick sanity checks you’ll care about =========
# a) No duplicate columns
print("\nDuplicate column names:", df_expanded.columns.duplicated().any())

# b) Any boolean columns that are near-constant (only True or only False among known rows)
near_constant = []
for _, r in col_summary[col_summary["dtype"]=="boolean"].iterrows():
    t, f = r.get("TRUE", 0), r.get("FALSE", 0)
    if r["non_null"] and min(t, f) / r["non_null"] < 0.01:
        near_constant.append((r["column"], t, f, r["non_null"]))
print("Near-constant boolean flags (<1% minority):", near_constant[:10])




In [27]:
import pandas as pd
import numpy as np

cols_df = pd.DataFrame({
    "order": np.arange(df_expanded.shape[1]),
    "column": df_expanded.columns,
    "dtype": df_expanded.dtypes.astype(str).values,
    "non_null": df_expanded.notna().sum().values,
})
cols_df  # displays every column



,order,column,dtype,non_null
0,0,business_id,object,52268
1,1,name,object,52268
2,2,address,object,51825
3,3,city,object,52268
4,4,state,object,52268
5,5,postal_code,object,52247
6,6,latitude,float64,52268
7,7,longitude,float64,52268
8,8,stars,float64,52268
9,9,review_count,int64,52268


In [28]:
df_expanded.shape   # -> (rows, columns)


(52268, 100)

In [31]:
import pandas as pd

# 1) Don’t truncate long strings
pd.set_option('display.max_colwidth', None)   # unlimited
pd.set_option('display.width', 200)           # wider table

# 2) See the full categories for the first 20 rows (adjust N as you like)
df_expanded[['business_id', 'name', 'categories']].head(20)


,business_id,name,categories
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,"Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries"
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,"Burgers, Fast Food, Sandwiches, Food, Ice Cream & Frozen Yogurt, Restaurants"
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,"Pubs, Restaurants, Italian, Bars, American (Traditional), Nightlife, Greek"
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,"Ice Cream & Frozen Yogurt, Fast Food, Burgers, Restaurants, Food"
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,"Vietnamese, Food, Restaurants, Food Trucks"
12,il_Ro8jwPlHresjw9EGmBg,Denny's,"American (Traditional), Restaurants, Diners, Breakfast & Brunch"
14,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,"Food, Delis, Italian, Bakeries, Restaurants"
15,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,"Sushi Bars, Restaurants, Japanese"
19,ROeacJQwBeh05Rqg7F6TCg,BAP,"Korean, Restaurants"
20,WKMJwqnfZKsAae75RMP6jA,Roast Coffeehouse and Wine Bar,"Coffee & Tea, Food, Cafes, Bars, Wine Bars, Restaurants, Nightlife"


In [32]:
#Build a clean list per business and inspect coverage

import pandas as pd
import re

def split_cats(s: str):
    if pd.isna(s): return []
    return [c.strip() for c in str(s).split(",") if c.strip()]

# add parsed list + count per row
df_expanded["categories_list"] = df_expanded["categories"].map(split_cats)
df_expanded["n_categories"] = df_expanded["categories_list"].str.len()

print("Rows:", len(df_expanded))
print("Non-null categories:", df_expanded["categories"].notna().sum())
print(df_expanded["n_categories"].describe())


Rows: 52268
Non-null categories: 52268
count    52268.000000
mean         4.660978
std          2.324649
min          1.000000
25%          3.000000
50%          4.000000
75%          6.000000
max         36.000000
Name: n_categories, dtype: float64


In [ ]:
#average business has ~4–5 labels (max 36 😅)

In [33]:
#What’s the vocabulary? How many distinct labels?
#Explode to one row per (business, category)
cats = (df_expanded[["business_id","categories_list"]]
          .explode("categories_list")
          .rename(columns={"categories_list":"category"})
          .dropna())

vocab_size = cats["category"].nunique()
print("Unique category labels:", vocab_size)

# Top 40 categories overall
top_cats = cats["category"].value_counts()
top_cats.head(40)


Unique category labels: 728


category
Restaurants                  52268
Food                         15472
Nightlife                     8723
Sandwiches                    8366
Bars                          8337
American (Traditional)        8139
Pizza                         7093
Fast Food                     6472
Breakfast & Brunch            6239
American (New)                6097
Burgers                       5636
Mexican                       4600
Italian                       4573
Coffee & Tea                  4053
Seafood                       3539
Chinese                       3169
Event Planning & Services     3068
Salad                         3064
Chicken Wings                 2966
Cafes                         2756
Delis                         2393
Caterers                      2059
Specialty Food                2011
Bakeries                      1889
Desserts                      1838
Japanese                      1830
Sports Bars                   1797
Sushi Bars                    1717
Barbeque   

In [34]:
#it’s a fixed catalog (728 distinct labels) and every row is a restaurant (the Restaurants label = 52,268).

In [35]:
# 40 least frequent categories (as a Series)
least_40 = top_cats.tail(40)
least_40


category
Climbing                          1
Parent Cafes                      1
Zoos                              1
Pet Photography                   1
Maternity Wear                    1
Recycling Center                  1
Guns & Ammo                       1
Limos                             1
Donburi                           1
Pressure Washers                  1
Travel Agents                     1
Fireworks                         1
Audio/Visual Equipment Rental     1
Crane Services                    1
RV Parks                          1
Pita                              1
Dry Cleaning                      1
Laundromat                        1
Guamanian                         1
Eyelash Service                   1
Eyewear & Opticians               1
Candle Stores                     1
Water Purification Services       1
Beach Bars                        1
Horse Racing                      1
Religious Items                   1
Pet Training                      1
Vacation Rentals   